# Intro

This notebook demonstrates how to use the retailoptimizer library to minimize the costs of your online purchases. 


In [1]:
import numpy as np
import pandas as pd
from retailoptimizer import RetailProblem, print_avail_solvers
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpInteger, LpBinary, makeDict

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../'))  
sys.path.insert(0, module_path)

In [ ]:
#Optional: test your installation of Pulp optimization libary
import pulp
pulp.pulpTestAll()

## Small sample problem using Python 

For a concrete example, assume a consumer wants to purchase fishing lures online. In the example below, the consumer wishes to buy 3 packs of plastic minnows and 20 fish hooks, and is considering 2 retailers. Let's find the cheapest purchase.

In the following python code, we create variables to define the problem. We must specify:
1. Items: What items we want to buy? (I.e. what fishing lures to buy?)
1. Quantities: How many of each item we want to buy? (E.g. we want to buy 20 hooks.)
1. Retailers: What are the names of the retailers we are shopping from?
1. Prices: What are the prices (per lure) for each fishing lure at each retailer?
1. Inventory: How many of each lure are in stock at each retailer?
   (If there is no inventory limit, enter a large value greater than the quantity desired, e.g. 100.)
1. Shipping: how much does shipping cost at each retailer, if your order is below the threshold for free shipping? (E.g., you must pay \\$4.00 for shipping at Retailer 2, unless you qualify for free shipping.)
1. Free shipping threshold: enter the thresholds (in dollars) to qualify for free shipping. (E.g. if your order at Retailer 2 is at least \\$60, you get free shipping.)
1. Sometimes you can reduce your bill by ordering more items than desired, to qualify for free shipping. Is that ok?

In [4]:
#Problem specification

#Order info
lures = ["plastic minnows", "hooks"]   
num_lures_to_buy = [3, 20]

#retailer info
retailers = ["Retailer1", "Retailer2"]  
prices = [
    [4.99, 5.49],   #price of plastic minnows at [Retailer1, Retailer2]
    [3.99, 3.49]    #price of hooks at [Retailer1, Retailer2]
]
inventory = [
    [100, 10],    #max number of plastic minnows available at [Retailer1, Retailer2]
    [15, 30],     #max number of hooks available at [Retailer1, Retailer2]
]

shipping = [7.0, 4.0] #shipping price in dollars at [Retailer1, Retailer2]
free_shipping_threshold = [50.0, 60.0] #Order threshold in dollars to qualify for free shipping @ [Retailer1, 2]

CAN_BUY_EXTRA_LURES_IF_CHEAPER = True

In [5]:
print("Available optimization solvers are: ")
print_avail_solvers()

Available optimization solvers are: 
GLPK_CMD
PULP_CBC_CMD
COIN_CMD


In [ ]:
#Specify solver to use

# SOLVER_NAME = 'PULP_CBC_CMD'
SOLVER_NAME = 'GLPK_CMD'

#Create Optimization Problem

p1 = RetailProblem(lures, num_lures_to_buy, retailers, prices,
                   inventory, shipping, free_shipping_threshold,
                   solver_name=SOLVER_NAME, 
                   can_buy_extra_lures_if_cheaper = CAN_BUY_EXTRA_LURES_IF_CHEAPER)

In [7]:
#Find optimimal solution
p1.solve()

solver = GLPK_CMD
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/cf78800b291a4f4992f58560f4e6c903-pulp.lp
 -o /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/cf78800b291a4f4992f58560f4e6c903-pulp.sol
Reading problem data from '/var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/cf78800b291a4f4992f58560f4e6c903-pulp.lp'...
10 rows, 8 columns, 22 non-zeros
8 integer variables, 4 of which are binary
44 lines were read
GLPK Integer Optimizer 5.0
10 rows, 8 columns, 22 non-zeros
8 integer variables, 4 of which are binary
Preprocessing...
4 constraint coefficient(s) were reduced
6 rows, 7 columns, 16 non-zeros
7 integer variables, 3 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.150e+02  ratio =  1.150e+02
GM: min|aij| =  5.433e-01  max|aij| =  1.841e+00  ratio =  3.388e+00
EQ: min|aij| =  2.952e-01  max|aij| =  1.000e+00  ratio =  3.388e+00
2N: min|aij| =  2.500e-01  max|aij| =  1.33

In [8]:
if p1.model.status == 1:
  print(f"Optimal solution found.")
  p1.print_optimization_results()
else:
  print(f"WARNING: optimal solution was not found. Model status was {LpStatus[p1.model.status]}")

Optimal solution found.
Status: Optimal
empty_order_retailer1 = 1
empty_order_retailer2 = 0
pay_shipping_retailer1 = 0
pay_shipping_retailer2 = 0
quant_hooks_retailer1 = 0
quant_hooks_retailer2 = 20
quant_plastic_minnows_retailer1 = 0
quant_plastic_minnows_retailer2 = 3
Total purchase cost =  86.27000000000001


## Results
In the results above (for this simple problem), we see it is cheapest to order everything from Retailer2, i.e. order 20 hooks and 3 plastic minnows from Retailer 2. We order nothing from Retailer 1. We also see that we do not need to pay for shipping. The total bill is \\$86.27

We can also programmatically access the quanties of items to order, e.g.

In [9]:
p1.quantity_to_order['hooks']['retailer1'].varValue

0

In [10]:
p1.quantity_to_order['hooks']['retailer2'].varValue

20

In [11]:
p1.quantity_to_order['plastic-minnows']['retailer1'].varValue

0

In [12]:
p1.quantity_to_order['plastic-minnows']['retailer2'].varValue

3

## Excel example
For every-day-use, it's easier to enter data in a spreadsheet than in Python data structures, so we illustrate how to solve the same problem using the sample Excel file included in the repository. You can also directly process the file from the command line.

In [13]:
EXCEL_INPUT_FILE = "example-problems/sample-order-small.xlsx"
EXCEL_OUTPUT_FILE = "example-problems/sample-order-small_results.xlsx"

p2 = RetailProblem.load_from_excel(EXCEL_INPUT_FILE)

/Users/michael/InstalledSoftware/anaconda3/envs/pulp/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [14]:
p2.initialize_optimization_problem()

In [15]:
p2.solve()

solver = None
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/michael/InstalledSoftware/anaconda3/envs/pulp/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/b09c3ba85fd244faba568629e78e8a2e-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/b09c3ba85fd244faba568629e78e8a2e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 60 RHS
At line 71 BOUNDS
At line 80 ENDATA
Problem MODEL has 10 rows, 8 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 84.77 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 5 rows, 7 columns (7 integer (3 of which binary)) and 15 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038

In [16]:
p2.print_optimization_results()

Status: Optimal
empty_order_retailer1 = 1.0
empty_order_retailer2 = 0.0
pay_shipping_retailer1 = 0.0
pay_shipping_retailer2 = 0.0
quant_hooks_retailer1 = 0.0
quant_hooks_retailer2 = 20.0
quant_plastic_minnows_retailer1 = 0.0
quant_plastic_minnows_retailer2 = 3.0
Total purchase cost =  86.27000000000001


In [17]:
if p2.model.status == 1:
  print(f"Optimal solution found. Saving results to {EXCEL_OUTPUT_FILE}")
  p2.save_results_to_excel(EXCEL_OUTPUT_FILE)
else:
  print(f"WARNING: optimal solution was not found. Model status was {LpStatus[p2.model.status]}")

Optimal solution found. Saving results to example-problems/sample-order-small_results.xlsx


In [18]:
EXCEL_OUTPUT_FILE

'example-problems/sample-order-small_results.xlsx'

You can similarly access results programatically

In [19]:
p2.quantity_to_order['hooks']['retailer1'].varValue

0.0

In [20]:
p2.quantity_to_order['hooks']['retailer2'].varValue

20.0

In [21]:
p2.quantity_to_order['plastic-minnows']['retailer1'].varValue

0.0

In [22]:
p2.quantity_to_order['plastic-minnows']['retailer2'].varValue

3.0